In [5]:
import nest_asyncio
nest_asyncio.apply()

from hddCRP.simulations import sim_markov, stack_distributions, interp_distributions, construct_default_hmm
from hddCRP.modelBuilder import cdCRP
import numpy as np
import pandas as pd

import arviz as az

import os
import sys

In [6]:
def generate_ps(sim, block):
    T_total = 25 * block;

    T_pre  = int(T_total/2)
    T_post = T_total - T_pre

    p_pre, t_pre, p_post, t_post = construct_default_hmm()
    
    
    sim = sim % 10

    match sim:
        case 0:
            return stack_distributions([T_pre, T_post], [p_pre, p_post]), p_pre
        case 1:
            return stack_distributions([T_pre, T_post], [t_pre, t_post]), p_pre
        case 2:
            return interp_distributions([T_total], [p_pre, p_post]), p_pre
        case 3:
            return interp_distributions([T_total], [t_pre, t_post]), p_pre
        case 4:
            return stack_distributions([T_total], [p_pre]), p_pre
        case 5:
            return stack_distributions([T_total], [t_pre]), p_pre
        case 6:
            return stack_distributions([T_total], [p_post]), p_post
        case 7:
            return stack_distributions([T_total], [t_post]), p_post

In [7]:
simulation_range = range(20,27+1)
block_range = range(1,4+1)
run_range = range(50)

results_directory = "Results/Simulations/"
OVERWRITE = False;


if(not os.path.exists(results_directory)):
    os.makedirs(results_directory)
num_responses = 3;

nback_depth   = 1;
context_depth = 1;
include_tau = True;

In [8]:
for simulation_id in simulation_range:
    for block_idx in block_range:

        print(f"BLOCK {block_idx}")
        obs_probs, init_obs_probs = generate_ps(simulation_id, block_idx)

        fit_file = f"{results_directory}/sim_{simulation_id}_block_{block_idx}"
        fit_summary_file = f"{results_directory}/sim_summary_{simulation_id}_block_{block_idx}"
        fit_file += f"_cd{context_depth}_nb{nback_depth}"
        fit_summary_file  += f"_cd{context_depth}_nb{nback_depth}"
        if(not include_tau):
            fit_file += f"_no_tau"
            fit_summary_file  += f"_no_tau"
        fit_file += f".pkl"
        fit_summary_file  += f".pkl"


        if(((not os.path.isfile(fit_file)) or (not os.path.isfile(fit_summary_file))) or OVERWRITE):
            simulation_fits = pd.DataFrame()
            simulation_fit_metrics = pd.DataFrame()
            for run_idx in run_range:
                sim_seed  = (simulation_id+1) * 10000 + nback_depth * 1001 + context_depth * 1000 + run_idx*100 
                stan_seed = (simulation_id+1) * 10000 + nback_depth * 1001 + context_depth * 1000 + run_idx*100 + block_idx
                sim_rng = np.random.Generator(np.random.MT19937(sim_seed))
                
                seqs = [sim_markov(obs_probs, init_obs_probs, rng=sim_rng)];
                subject_labels = [0];
                session_labels_all = ["A"];
                print(seqs)
                print(f"BLOCK {block_idx} - RUN {run_idx}")

                model = cdCRP(seqs, subject_labels=subject_labels, session_labels=session_labels_all, possible_observations=list(range(num_responses)));
                model.same_nback_depth = nback_depth;
                model.context_depth = context_depth;
                model.within_session_decay_enabled = include_tau
                
                model.build(random_seed=stan_seed);
                model.fit_model()

                fit_df  = model.fit.to_frame()
                # map_fit = model.get_map()
                fit_df["block"] = block_idx
                fit_df["run"]   = run_idx
                fit_df["simulation"]   = simulation_id
                summary_df = model.fit_summary()
                summary_df["block"] = block_idx
                summary_df["run"]   = run_idx
                # summary_df["MAP"] = pd.Series(map_fit)
                summary_df["simulation"]   = simulation_id

                

                simulation_fit_metrics = pd.concat([simulation_fit_metrics,summary_df], copy=False)
                simulation_fits = pd.concat([simulation_fits,fit_df], copy=False)

            simulation_fits.to_pickle(fit_file)
            simulation_fit_metrics.to_pickle(fit_summary_file)
        else:
            print("Fit files found: not overriding")


BLOCK 1
[array([2, 2, 0, 1, 2, 1, 2, 0, 0, 0, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 0, 1,
       0, 1, 1])]
BLOCK 1 - RUN 0
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   1% (100/8000)


Sampling:   2% (200/8000)
Sampling:   4% (300/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  25% (2000/8000)
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 0.000883 seconds
  1000 transitions using 10 leapfrog steps per transition would take 8.83 seconds.
  Adjust your expectations accordingly!
  Informational Message: The current Metropolis proposal is about to be rejected because of the following issue:
  Exception: bernoulli_lpmf: Probability parameter[1] is -nan, but must be in the interval [0, 1] (in '/tmp/httpstan_ym7rrkw0/model_buyrpbfc.stan', line 142, column 4 to column 22)
  If this warning occurs sporadically, such as for highly constrained variable types like covariance matrices, then the sampler is fine,
  but if this warning occurs often then your model may be either severely ill-conditioned or misspecified.
  

[array([0, 2, 0, 2, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 2, 1])]
BLOCK 1 - RUN 1
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   1% (102/8000)
Sampling:   3% (202/8000)
Sampling:   6% (501/8000)
Sampling:  11% (900/8000)
Sampling:  16% (1300/8000)
Sampling:  20% (1600/8000)
Sampling:  25% (2000/8000)
Sampling:  30% (2400/8000)
Sampling:  34% (2700/8000)
Sampling:  39% (3100/8000)
Sampling:  55% (4400/8000)
Sampling:  70% (5600/8000)
Sampling:  85% (6800/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/

...


Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampl

[array([0, 2, 2, 0, 2, 0, 1, 0, 1, 1, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2,
       0, 1, 0, 2, 2, 2, 2, 1, 0, 2, 2, 2, 1, 0, 0, 1, 1, 1, 0, 1, 2, 2,
       1, 1, 0, 2, 1, 2, 1, 0, 1, 1, 0, 1, 2, 2, 1, 0, 1, 2, 0, 2, 2, 1,
       1, 1, 2, 2, 1, 1, 0, 1, 1])]
BLOCK 3 - RUN 6
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (401/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampl

[array([1, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 1, 2, 1, 0, 0, 0, 2, 1, 1, 1, 2,
       1, 2, 1, 0, 0, 1, 1, 1, 2, 0, 1, 2, 2, 2, 2, 1, 1, 0, 0, 1, 2, 1,
       2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 0, 2, 1, 0,
       2, 2, 1, 1, 1, 1, 1, 0, 1])]
BLOCK 3 - RUN 7
Building...



Building: found in cache, done.Messages from stanc:
    means either no prior is provided, or the prior(s) depend on data
    variables. In the later case, this may be a false positive.
    prior is provided, or the prior(s) depend on data variables. In the later
    case, this may be a false positive.
    either no prior is provided, or the prior(s) depend on data variables. In
    the later case, this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  15% (1200/8000)
Sampling:  16% (1300/8000)
Sampl